In [1]:
import argparse
import logging
import os
from datetime import datetime, timedelta
from os import cpu_count

from download import download_sdo as sdo
import flare_event_downloader as fed
import sxr_downloader as sxr

/home/griffingoodwin/anaconda3/envs/down/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import argparse
import logging
import os
from datetime import datetime, timedelta
from os import cpu_count

from download import download_sdo as sdo
import flare_event_downloader as fed
import sxr_downloader as sxr

class FlareDownloadProcessor:
    def __init__(self, FlareEventDownloader, SDODownloader, SXRDownloader):
        """
        Initialize the FlareDownloadProcessor.
        This class is responsible for processing AIA flare downloads.
        """
        self.FlareEventDownloader = FlareEventDownloader
        self.SDODownloader = SDODownloader
        self.SXRDownloader = SXRDownloader

    def process_download(self, time_before_start=timedelta(minutes=60), time_after_end=timedelta(minutes=0)):

        fl_events = self.FlareEventDownloader.download_events()
        print(fl_events)
        [os.makedirs(os.path.join(self.SDODownloader.ds_path, str(c)), exist_ok=True) for c in
         [94, 131, 171, 193, 211, 304]]
        for i, events in enumerate(fl_events.iterrows()):
            event = events[1]
            start_time = event['event_starttime'] - time_before_start
            end_time = event['event_endtime'] + time_after_end
            self.SXRDownloader.download_and_save_goes_data(start_time.strftime('%Y-%m-%d'),
                                                           end_time.strftime('%Y-%m-%d'), max_workers= os.cpu_count())
            processed_dates = set()
            for d in [start_time + i * timedelta(minutes=1) for i in
                      range((end_time - start_time) // timedelta(minutes=1))]:
                # Only download if we haven't processed this date yet
                if d.isoformat() not in processed_dates:
                    self.SDODownloader.downloadDate(d)
                    processed_dates.add(d.isoformat())
            logging.info(f"Processed flare event {i + 1}/{len(fl_events)}: {event['event_starttime']} to {event['event_endtime']}")


if __name__ == '__main__':
    # parser = argparse.ArgumentParser(description='Download SDO data from JSOC with quality check and fallback for flare data')
    # parser.add_argument('--download_dir', type=str, help='path to the download directory.')
    # parser.add_argument('--email', type=str, help='registered email address for JSOC.')
    # parser.add_argument('--start_date', type=str, help='start date in format YYYY-MM-DD.')
    # parser.add_argument('--end_date', type=str, help='end date in format YYYY-MM-DD.', required=False,
    #                      default=str(datetime.now()).split(' ')[0])
    # parser.add_argument('--time_before_start', type=int, help='', required=False)
    #
    # args = parser.parse_args()
    # download_dir = args.download_dir
    # start_date = args.start_date
    # end_date = args.end_date
    # cadence = args.cadence

    sxr_downloader = sxr.SXRDownloader("/mnt/data/GOES-additional", "/mnt/data/GOES-additional/combined")
    flare_event = fed.FlareEventDownloader("2012-01-01", "2020-01-01", event_type="FL", GOESCls="M1.0", directory="/mnt/data/SDO-AIA-additional/FlareEvents")
    sdo_downloader = sdo.SDODownloader("/mnt/data/SDO-AIA-additional","ggoodwin5@gsu.edu")
    FlareDownloadProcessor(flare_event, sdo_downloader,sxr_downloader).process_download()

In [2]:
flare_event2 = fed.FlareEventDownloader("2024-01-01", "2024-03-01", event_type="FL", GOESCls="M1.0", directory="/mnt/data/SDO-AIA-additional/FlareEvents")

In [3]:
flare_event2.download_events()

Available columns: ['gs_thumburl', 'comment_count', 'hpc_bbox', 'frm_humanflag', 'hgc_coord', 'obs_levelnum', 'hpc_coord', 'event_npixels', 'gs_imageurl', 'ar_polarity', 'frm_paramset', 'hrc_coord', 'event_starttime', 'ar_mtwilsoncls', 'event_type', 'intensmin', 'fl_fluence', 'obs_meanwavel', 'frm_url', 'skel_chaincode', 'bound_chaincode', 'noposition', 'active', 'intensmax', 'frm_versionnumber', 'fl_halphaclass', 'area_uncert', 'obs_dataprepurl', 'hpc_geom', 'hgc_bbox', 'intensmedian', 'chaincodetype', 'obs_channelid', 'event_clippedspatial', 'ar_noaaclass', 'SOL_standard', 'event_avg_rating', 'eventtype', 'hpc_boundcc', 'event_mapurl', 'frm_contact', 'ar_penumbracls', 'intensmean', 'bound_ccstartc1', 'frm_name', 'area_atdiskcenter', 'frm_identifier', 'obs_observatory', 'event_description', 'boundbox_c2ur', 'obs_firstprocessingdate', 'boundbox_c2ll', 'frm_institute', 'hrc_bbox', 'refs_orig', 'ar_mcintoshcls', 'event_maskurl', 'bound_ccstartc2', 'gs_movieurl', 'event_score', 'skel_star

,event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum,hgc_coord.lon,hgc_coord.lat,hgc_coord.obstime.jd1,hgc_coord.obstime.jd2,hgc_coord.observer.lon,hgc_coord.observer.lat,hgc_coord.observer.radius,hgc_coord.observer.obstime.jd1,hgc_coord.observer.obstime.jd2
0,2024-01-01 08:33:00,2024-01-01 08:54:00,2024-01-01 09:04:00,M2.3,13536,222.77349,0.0,2460311.0,-0.143750,0.0,-2.981678,0.983315,2460311.0,-0.143750
1,2024-01-01 11:54:00,2024-01-01 12:25:00,2024-01-01 12:35:00,M4.7,13536,220.93509,0.0,2460311.0,-0.004167,0.0,-2.998047,0.983313,2460311.0,-0.004167
2,2024-01-02 18:02:00,2024-01-02 18:30:00,2024-01-02 18:56:00,M1.1,13536,204.39887,0.0,2460312.0,0.251389,0.0,-3.144465,0.983307,2460312.0,0.251389
3,2024-01-04 01:08:00,2024-01-04 01:16:00,2024-01-04 01:22:00,M1.1,13536,187.33258,0.0,2460314.0,-0.452778,0.0,-3.293964,0.983310,2460314.0,-0.452778
4,2024-01-04 01:22:00,2024-01-04 01:55:00,2024-01-04 02:12:00,M3.8,13536,187.20454,0.0,2460314.0,-0.443056,0.0,-3.295080,0.983310,2460314.0,-0.443056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2024-02-24 10:32:00,2024-02-24 10:57:00,2024-02-24 11:03:00,M2.2,13590,230.62775,0.0,2460365.0,-0.061111,0.0,-7.123890,0.989512,2460365.0,-0.061111
78,2024-02-24 11:03:00,2024-02-24 11:18:00,2024-02-24 11:25:00,M3.6,13590,230.34422,0.0,2460365.0,-0.039583,0.0,-7.124404,0.989516,2460365.0,-0.039583
79,2024-02-24 11:54:00,2024-02-24 11:59:00,2024-02-24 12:04:00,M1.0,13590,229.87776,0.0,2460365.0,-0.004167,0.0,-7.125247,0.989524,2460365.0,-0.004167
80,2024-02-25 16:47:00,2024-02-25 17:22:00,2024-02-25 17:47:00,M2.0,13590,214.02721,0.0,2460366.0,0.199306,0.0,-7.152234,0.989800,2460366.0,0.199306
